In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install dgl
!pip install torch_geometric
!pip install networkx==2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 23.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=53be1ddcf46fd4ae0c01c81d12c21f25c501b124a352bd3eededc750bac5073b
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.2 MB/s eta 0:00:00
  Attempting unins

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import string
import networkx as nx
import nltk

import tqdm
import en_core_web_lg
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import spacy
from time import time
import dgl
from scipy import sparse


import torch
from torch_geometric.data import Data, HeteroData
from datetime import datetime


from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D



DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Load tweet file

In [ ]:
p_part1 = '68841_tweets_multiclasses_filtered_0722_part1.npy'
p_part2 = '68841_tweets_multiclasses_filtered_0722_part2.npy'


# Concatenating The Dataset

In [ ]:
df_np_part1 = np.load(p_part1, allow_pickle=True)
df_np_part2 = np.load(p_part2, allow_pickle=True)
df_np = np.concatenate((df_np_part1, df_np_part2), axis = 0)
print("[INFO] Data Loaded...")


df = pd.DataFrame(data=df_np, columns=["event_id", "tweet_id", "text", "user_id", "created_at", "user_loc",\
    "place_type", "place_full_name", "place_country_code", "hashtags", "user_mentions", "image_urls", "entities", 
    "words", "filtered_words", "sampled_words"])

print("[INFO] Data converted to dataframe...")



[INFO] Data Loaded...
[INFO] Data converted to dataframe...


In [ ]:
print(df.shape)

(68841, 16)


In [ ]:
df.head(10)

,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words
0,0,256292946331181056,Nobel prize in literature to be announced http...,47667947,2012-10-11 07:19:34,"Munich, Germany",,,,[],[],[],[],"[literature, Nobel, prize, announce]","[literature, nobel, prize, announce]",[]
1,0,256333064467279872,“@marvicleonen: Is it true that UP won UAAP ba...,67518107,2012-10-11 09:58:59,Philippines,,,,[],[28775032],[],"[(UP, ORG), (Next year, DATE), (Dean, PERSON)]","[Dean, Sure, year, yan, na, "", basketball, tru...","[dean, sure, year, yan, na, basketball, true, ...",[]
2,0,256334302034399232,"Congrats, Ateneo! Last na yan ha. Season 76 wi...",97449266,2012-10-11 10:03:54,,,,,[],[],[],"[(Ateneo, PERSON), (Season 76, PERSON)]","[yan, ☺, na, ha, different, last, Ateneo, cong...","[yan, na, ha, different, last, ateneo, congrat...",[]
3,0,256335853738160128,"""@SMARTPromos: SMART never wants you to be lef...",405138197,2012-10-11 10:10:04,Lost in Dreamland,,,,[],[106915372],[],"[(BIG, ORG), (BIG, ORG)]","[never, s, yan, next, na, thing, Ano, leave, t...","[never, yan, next, na, thing, ano, leave, that...",[]
4,0,256346272506712064,CCTV invite hints at Nobel literature prize fo...,197326414,2012-10-11 10:51:28,Taiwan(R.O.C),,,,[],[],[],"[(CCTV, ORG), (Nobel, WORK_OF_ART), (Mo Yan, P...","[invite, prize, Yan, literature, hint, CCTV, N...","[invite, prize, yan, literature, hint, cctv, n...",[]
5,0,256346650132508673,mjzone58: SIR HINDI BYAHE YAN. WALA TALAGANG P...,397051720,2012-10-11 10:52:58,,,,,[],[],[],"[(WALA, ORG)]","[HINDI, wala, yan, mjzone58, panalo, SIR, BYAH...","[hindi, wala, yan, panalo, sir, byahe, talagan...","[panalo, byahe]"
6,0,256349023773024256,BREAKING: The 2012 Nobel Prize for Literature ...,7424642,2012-10-11 11:02:24,Tokyo,,,,[],[],[],"[(Nobel Prize for Literature, WORK_OF_ART), (C...","[award, author, Yan, BREAKING, chinese, Litera...","[award, author, yan, breaking, chinese, litera...",[]
7,0,256349178991624192,"""@GuardianBooks: And the #Nobel prize for lite...",24515914,2012-10-11 11:03:01,Montevideo,,,,[Nobel],[22001973],[],"[(China, GPE)]","[prize, Chinas, literature, Yan, go, Mo]","[prize, chinas, literature, yan, go, mo]",[]
8,0,256349191863943168,Chinese author Mo Yan wins Nobel prize for lit...,14860646,2012-10-11 11:03:04,Nepal,,,,[],[],[],"[(Chinese, NORP), (Mo Yan, PERSON), (Nobel, WO...","[author, prize, win, literature, Nobel, chines...","[author, prize, win, literature, nobel, chines...",[]
9,0,256349325876137984,Congratulations! Mo Yan of this year Nobel Prize!,489617283,2012-10-11 11:03:36,,,,,[],[],[],"[(Mo Yan, PERSON), (this year, DATE), (Nobel P...","[year, Yan, Congratulations, Prize, Nobel, Mo]","[year, yan, congratulations, prize, nobel, mo]",[]


In [ ]:
df.text = df.text.str.strip()


In [ ]:
df["text"]

0        Nobel prize in literature to be announced http...
1        “@marvicleonen: Is it true that UP won UAAP ba...
2        Congrats, Ateneo! Last na yan ha. Season 76 wi...
3        "@SMARTPromos: SMART never wants you to be lef...
4        CCTV invite hints at Nobel literature prize fo...
                               ...                        
68836    Oh my god it's like the World Series all over ...
68837    Looks like the yankees are winning the world s...
68838    The Yankees are going to win the world Series ...
68839    Cubs last World Series win, they were keeping ...
68840    I hate to say this but the giants are going to...
Name: text, Length: 68841, dtype: object

In [ ]:
df.dtypes

event_id                      object
tweet_id                      object
text                          object
user_id                       object
created_at            datetime64[ns]
user_loc                      object
place_type                    object
place_full_name               object
place_country_code            object
hashtags                      object
user_mentions                 object
image_urls                    object
entities                      object
words                         object
filtered_words                object
sampled_words                 object
dtype: object

In [ ]:
df.isnull().sum()

event_id              0
tweet_id              0
text                  0
user_id               0
created_at            0
user_loc              0
place_type            0
place_full_name       0
place_country_code    0
hashtags              0
user_mentions         0
image_urls            0
entities              0
words                 0
filtered_words        0
sampled_words         0
dtype: int64

# Defining preprocessing functions for TEXT

In [ ]:

def clean_text(text):
    # Convert text to lowercase
    text = str(text).lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove digits
    text = re.sub(r'\d+', '', text)
    return text

def tokenize_text(text):
    # Tokenize text using TweetTokenizer
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def lemmatize_tokens(tokens):
    # Lemmatize tokens using WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

def remove_stopwords(tokens):
    # Remove stop words using NLTK's English stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

In [ ]:
# Apply preprocessing to the text column
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].apply(tokenize_text)
df['text'] = df['text'].apply(lemmatize_tokens)
df['text'] = df['text'].apply(remove_stopwords)


In [ ]:
df.head()

,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words
0,0,256292946331181056,"[nobel, prize, literature, announced]",47667947,2012-10-11 07:19:34,"Munich, Germany",,,,[],[],[],[],"[literature, Nobel, prize, announce]","[literature, nobel, prize, announce]",[]
1,0,256333064467279872,"[“, marvicleonen, true, uaap, basketball, ”, n...",67518107,2012-10-11 09:58:59,Philippines,,,,[],[28775032],[],"[(UP, ORG), (Next year, DATE), (Dean, PERSON)]","[Dean, Sure, year, yan, na, "", basketball, tru...","[dean, sure, year, yan, na, basketball, true, ...",[]
2,0,256334302034399232,"[congrats, ateneo, last, na, yan, ha, season, ...",97449266,2012-10-11 10:03:54,,,,,[],[],[],"[(Ateneo, PERSON), (Season 76, PERSON)]","[yan, ☺, na, ha, different, last, Ateneo, cong...","[yan, na, ha, different, last, ateneo, congrat...",[]
3,0,256335853738160128,"[smartpromos, smart, never, want, left, behind...",405138197,2012-10-11 10:10:04,Lost in Dreamland,,,,[],[106915372],[],"[(BIG, ORG), (BIG, ORG)]","[never, s, yan, next, na, thing, Ano, leave, t...","[never, yan, next, na, thing, ano, leave, that...",[]
4,0,256346272506712064,"[cctv, invite, hint, nobel, literature, prize,...",197326414,2012-10-11 10:51:28,Taiwan(R.O.C),,,,[],[],[],"[(CCTV, ORG), (Nobel, WORK_OF_ART), (Mo Yan, P...","[invite, prize, Yan, literature, hint, CCTV, N...","[invite, prize, yan, literature, hint, cctv, n...",[]


# Convert Date and Time in Numeric

In [ ]:
# sort data in DATAFRAME 1 by time
df = df.sort_values(by='created_at').reset_index()
# append date
df['date'] = [d.date() for d in df['created_at']]


In [ ]:
# df['created_at'] = pd.to_datetime(df['created_at'])
#
# # convert datetime column to numeric format
# df['created_at'] = pd.to_numeric(df['created_at'].dt.strftime('%Y%m%d%H%M%S'))

In [ ]:
df.head()

,index,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words,date
0,61124,394,255819992157786112,"[hiphop, award, bout, live]",250870763,2012-10-10 00:00:13,,,,,[],[],[],[],"[award, live, bout, hiphop]","[award, live, bout, hiphop]",[],2012-10-10
1,61125,394,255820118095978496,"[hiphop, award, time]",28026779,2012-10-10 00:00:43,SoundCloud/RaRaSupaStar,,,,[],[],[],[],"[HIPHOP, AWARDS, time]","[hiphop, awards, time]",[],2012-10-10
2,61126,394,255820147489636353,"[bet, hiphop, award]",566825483,2012-10-10 00:00:50,,,,,[],[],[],"[(Bet, GPE)]","[award, bet, hiphop]","[award, bet, hiphop]",[],2012-10-10
3,61127,394,255820164023595008,"[bet, hiphop, award]",197834311,2012-10-10 00:00:54,Saint Lucia ☀️🌴🇱🇨,,,,[],[],[],[],"[HipHop, BET, award]","[hiphop, bet, award]",[],2012-10-10
4,61128,394,255820180884701184,"[watchin, da, bet, hiphop, award]",439490861,2012-10-10 00:00:58,"Michigan, USA",,,,[],[],[],[],"[Hiphop, Watchin, Awards, Da, BET]","[hiphop, watchin, awards, da, bet]",[],2012-10-10


# Load message file

In [ ]:
load_path = 'all_df_words_ents_mids.npy'

In [ ]:
df_np = np.load(load_path, allow_pickle=True)

print("[INFO] Data Loaded...")

df1 = pd.DataFrame(data=df_np, \
    columns=['document_ids', 'sentence_ids', 'sentences', 'event_type_ids', 'words', 'unique_words', 'entities', 'message_ids'])

print("[INFO] Data converted to dataframe...")


[INFO] Data Loaded...
[INFO] Data converted to dataframe...


In [ ]:
print(df1.shape)

(10242, 8)


In [ ]:
df1.head()

,document_ids,sentence_ids,sentences,event_type_ids,words,unique_words,entities,message_ids
0,387fe1dfe55067eb29e1fd4116d37af3,14,"According to author Neill Macaulay, ""he would ...",23,"[accord, author, Neill, Macaulay, would, attac...","[odd, heavily, firepower, cover, orwhen, clear...","[(Neill Macaulay, PERSON)]",387fe1dfe55067eb29e1fd4116d37af3_14
1,387fe1dfe55067eb29e1fd4116d37af3,10,Sandino had a tendency to greatly exaggerate n...,37,"[sandino, tendency, greatly, exaggerate, numbe...","[probably, inaccurate, exaggerate, sandino, re...","[(Sandino, ORG), (60, CARDINAL)]",387fe1dfe55067eb29e1fd4116d37af3_10
2,268c4763208c87ed7ebf55565c274d23,4,British forces in India were considerably stro...,42,"[british, force, India, considerably, strong, ...","[strong, Cornwallis, RearAdmiral, suppoed, Wil...","[(British, NORP), (India, GPE), (French, NORP)...",268c4763208c87ed7ebf55565c274d23_4
3,268c4763208c87ed7ebf55565c274d23,8,British forces constructed trenches and batter...,44,"[british, force, construct, trench, battery, o...","[construct, often, heavy, fire, force, british...","[(British, NORP), (the following weeks, DATE)]",268c4763208c87ed7ebf55565c274d23_8
4,268c4763208c87ed7ebf55565c274d23,0,"""For other sieges with this name, see Siege of...",45,"[siege, name, see, Siege, Pondicherry, disambi...","[Siege, early, stage, military, French, operat...","[(Siege, FAC), (Pondicherry, GPE), (The Siege ...",268c4763208c87ed7ebf55565c274d23_0


# Preprocessing

In [ ]:
df1.isnull().sum()

document_ids      0
sentence_ids      0
sentences         0
event_type_ids    0
words             0
unique_words      0
entities          0
message_ids       0
dtype: int64

In [ ]:
df1.dtypes

document_ids      object
sentence_ids      object
sentences         object
event_type_ids    object
words             object
unique_words      object
entities          object
message_ids       object
dtype: object

# Convert ids into Numeric

In [ ]:
# Create a dictionary that maps unique string values of 'document_ids' to integers
doc_id_map = {  doc_id:i for i, doc_id in enumerate(df1['document_ids'].unique())}

# Map the 'document_ids' column to integer values
df1['document_ids'] = df1['document_ids'].map(doc_id_map)

# Convert the 'document_ids' column to integer type
df1['document_ids'] = df1['document_ids'].astype(int)

In [ ]:
# Convert the 'sentence_ids' column to integer type
df1['sentence_ids'] = df1['sentence_ids'].astype(int)

In [ ]:
# Create a dictionary that maps unique string values of 'sentence_ids' to integers
msg_id_map = {msg_id:i for i, msg_id in enumerate(df1['message_ids'].unique())}

# Map the 'sentence_ids' column to integer values
df1['message_ids'] = df1['message_ids'].map(msg_id_map)

# Convert the 'sentence_ids' column to integer type
df1['message_ids'] = df1['message_ids'].astype(int)

In [ ]:
# Apply preprocessing to the text column
df1['sentences'] = df1['sentences'].apply(clean_text)
df1['sentences'] = df1['sentences'].apply(tokenize_text)
df1['sentences'] = df1['sentences'].apply(lemmatize_tokens)
df1['sentences'] = df1['sentences'].apply(remove_stopwords)


In [ ]:
df1.head()

,document_ids,sentence_ids,sentences,event_type_ids,words,unique_words,entities,message_ids
0,0,14,"[according, author, neill, macaulay, would, at...",23,"[accord, author, Neill, Macaulay, would, attac...","[odd, heavily, firepower, cover, orwhen, clear...","[(Neill Macaulay, PERSON)]",0
1,0,10,"[sandino, tendency, greatly, exaggerate, numbe...",37,"[sandino, tendency, greatly, exaggerate, numbe...","[probably, inaccurate, exaggerate, sandino, re...","[(Sandino, ORG), (60, CARDINAL)]",1
2,1,4,"[british, force, india, considerably, stronger...",42,"[british, force, India, considerably, strong, ...","[strong, Cornwallis, RearAdmiral, suppoed, Wil...","[(British, NORP), (India, GPE), (French, NORP)...",2
3,1,8,"[british, force, constructed, trench, battery,...",44,"[british, force, construct, trench, battery, o...","[construct, often, heavy, fire, force, british...","[(British, NORP), (the following weeks, DATE)]",3
4,1,0,"[siege, name, see, siege, pondicherry, disambi...",45,"[siege, name, see, Siege, Pondicherry, disambi...","[Siege, early, stage, military, French, operat...","[(Siege, FAC), (Pondicherry, GPE), (The Siege ...",4


# FILTERING TWEETS

In [ ]:
df.tail()

,index,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words,date
68836,28486,67,265966361803640832,"[“, ftofreal, state, might, legalize, marijuan...",257380081,2012-11-06 23:58:16,2 5 2,,,,[],[],[],"[(3, CARDINAL)]","[state, legalize, may, "", marijuana, nc, chang...","[state, legalize, may, marijuana, nc, change, ...",[],2012-11-06
68837,24465,52,265966512672759808,"[im, watching, united, state, presidential, el...",115979133,2012-11-06 23:58:52,,,,,"[GetGlue, election2012]",[],[],"[(United States Presidential Election 2012, OR...","[be, United, -PRON-, other, Election, checkedi...","[be, united, other, election, checkedin, state...",[checkedin],2012-11-06
68838,24466,52,265966529496092675,"[watch, election, presentation]",283768568,2012-11-06 23:58:56,,,,,[],[],[],[],"[election, watch, presentation]","[election, watch, presentation]",[],2012-11-06
68839,46775,197,265966617589084160,"[bombing, rock, damascus, brother, parliament,...",311802961,2012-11-06 23:59:17,,,,,[],[],[],"[(Damascus, GPE)]","[Anything, Damascus, rock, kill, brother, co, ...","[anything, damascus, rock, kill, brother, co, ...",[],2012-11-06
68840,63652,430,265966797839273984,"[romney, win, idk, could, leave, country, cuz,...",432194703,2012-11-07 00:00:00,,,,,[],[],[],[],"[not, cuz, afford, idk, country, can, whole, w...","[not, cuz, afford, idk, country, can, whole, w...",[],2012-11-07


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# Check if a GPU is available and use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained sentiment analysis model and tokenizer
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Create the sentiment analysis pipeline
sentiment_analyzer = TextClassificationPipeline(model=model, tokenizer=tokenizer, task='sentiment-analysis')

In [ ]:
sentiment_analyzer(df.text[68839])[0]['score']

0.9992014765739441

In [ ]:
messages =  df.text

In [ ]:
# Compute the sentiment scores for each message
batch_size = 64
sentiment_scores = []
for i in tqdm.tqdm(range(0, len(messages), batch_size)):
    batch = messages[i:i + batch_size].apply(str).tolist()
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1).cpu().numpy()

    for probability in probabilities:
        sentiment_scores.append(probability[1])  # Get the score for the positive sentiment class


# Add sentiment scores to the DataFrame
df['sentiment_scores'] = sentiment_scores

100%|██████████| 1076/1076 [2:28:49<00:00,  8.30s/it]


In [ ]:
# df.to_csv("processed_tweets_data.csv", index=False)

In [ ]:
new_DF = pd.read_csv("processed_tweets_data.csv")
new_DF['sentiment_scores'].tail()

<ipython-input-40-8dc1be865af1>:1: DtypeWarning: Columns (0,4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  new_DF = pd.read_csv("processed_tweets_data.csv")


68842    0.024878
68843    0.012188
68844    0.949300
68845    0.001910
68846    0.017375
Name: sentiment_scores, dtype: float64

In [ ]:
threshold = 0.5

filtered_df = new_DF[new_DF['sentiment_scores'] >= threshold]
filtered_df.shape

(21311, 19)

In [ ]:
print("TOTAL MESSAGES: ", len(new_DF))
print("FILTERED MESSAGES:", len(filtered_df))

TOTAL MESSAGES:  68847
FILTERED MESSAGES: 21311


In [ ]:
# # Set batch size and preallocate list for sentiment scores
# batch_size = 256
# sentiment_labels = []

# # Process messages in batches
# for i in tqdm.tqdm(range(0, len(messages), batch_size)):
#     batch = messages[i:i + batch_size].apply(str).tolist()
#     results = sentiment_analyzer(batch)
#     for result in results:
#         sentiment_labels.append(result['label'])

# # Add sentiment scores to the DataFrame
# df['sentiment_labels'] = sentiment_labels

In [ ]:
filtered_df.drop(['sentiment_scores'], axis=1, inplace=True)

<ipython-input-44-aa0aeb16561d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['sentiment_scores'], axis=1, inplace=True)


# Generating the initial message features

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D
import numpy as np

# Parameters for LSTM and preprocessing
max_words = 10000
max_sequence_length = 100
embedding_dim = 300


# Tokenize the text data and convert them into sequences
def tokenize_text(df):
    tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(df['filtered_words'].apply(lambda x: ' '.join(x)).values)
    sequences = tokenizer.texts_to_sequences(df['filtered_words'].apply(lambda x: ' '.join(x)).values)
    word_index = tokenizer.word_index
    return pad_sequences(sequences, maxlen=max_sequence_length), word_index

# Load the pre-trained word embeddings and create the embedding matrix
def create_embedding_matrix(word_index, nlp):
    embedding_matrix = np.zeros((max_words, embedding_dim))
    for word, i in word_index.items():
        if i < max_words:
            token = nlp.vocab.get_vector(word)
            if token is not None:
                embedding_matrix[i] = token
    return embedding_matrix

# Create an efficient bidirectional LSTM model for generating message features
def create_lstm_model(embedding_matrix):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
    model.add(Bidirectional(LSTM(embedding_dim, return_sequences=True)))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    return model


# Generate the message features using the LSTM model
def documents_to_features(df, nlp, word_index):
    sequences, _ = tokenize_text(df)
    embedding_matrix = create_embedding_matrix(word_index, nlp)
    model = create_lstm_model(embedding_matrix)
    features = model.predict(sequences)
    return features





# '''
# The embedding of each document is the average of the pre-trained embeddings of all the words in it.
# '''

# def get_document_embeddings(df):
#     # Load the pre-trained English language model from Spacy library
#     nlp = en_core_web_lg.load()
#     # Apply the language model on each document and calculate its vector representation
#     embeddings = df.filtered_words.apply(lambda x: nlp(' '.join(x)).vector).values
#     # Stack all the embeddings into a numpy array and return it
#     return np.stack(embeddings, axis=0)

'''
This function encodes a timestamp string in the format '2012-10-11 07:19:34' into a list of two time features.
'''

def extract_time_features(timestamp_str):
    # Convert the timestamp string to a datetime object
    timestamp = datetime.fromisoformat(str(timestamp_str))
    # Define a datetime object to represent the zero time for OLE time
    OLE_TIME_ZERO = datetime(1899, 12, 30)
    # Calculate the time difference between the timestamp and the OLE time zero
    delta = timestamp - OLE_TIME_ZERO
    # Calculate the time features by normalizing the time difference into fractions of a day
    time_features = [(float(delta.days) / 100000.), (float(delta.seconds) / 86400)] # 86,400 seconds in a day
    # Return the time features as a list
    return time_features

'''
This function encodes the timestamps of all the messages in the dataframe into a numpy array of time features.
'''
def get_time_features(df):
    # Apply the extract_time_features function on each timestamp string in the dataframe
    time_features = np.asarray([extract_time_features(timestamp_str) for timestamp_str in df['created_at']])
    # Return the time features as a numpy array
    return time_features


In [ ]:
filtered_df.head()

,index,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words,date
0,61124.0,394.0,2.558200e+17,"['hiphop', 'award', 'bout', 'live']",250870763,2012-10-10 00:00:13,NaN,NaN,NaN,NaN,[],[],[],[],"['award', 'live', 'bout', 'hiphop']","['award', 'live', 'bout', 'hiphop']",[],2012-10-10
1,61125.0,394.0,2.558201e+17,"['hiphop', 'award', 'time']",28026779,2012-10-10 00:00:43,SoundCloud/RaRaSupaStar,NaN,NaN,NaN,[],[],[],[],"['HIPHOP', 'AWARDS', 'time']","['hiphop', 'awards', 'time']",[],2012-10-10
2,61126.0,394.0,2.558201e+17,"['bet', 'hiphop', 'award']",566825483,2012-10-10 00:00:50,NaN,NaN,NaN,NaN,[],[],[],"[('Bet', 'GPE')]","['award', 'bet', 'hiphop']","['award', 'bet', 'hiphop']",[],2012-10-10
3,61127.0,394.0,2.558202e+17,"['bet', 'hiphop', 'award']",197834311,2012-10-10 00:00:54,Saint Lucia ☀️🌴🇱🇨,NaN,NaN,NaN,[],[],[],[],"['HipHop', 'BET', 'award']","['hiphop', 'bet', 'award']",[],2012-10-10
4,61128.0,394.0,2.558202e+17,"['watchin', 'da', 'bet', 'hiphop', 'award']",439490861,2012-10-10 00:00:58,"Michigan, USA",NaN,NaN,NaN,[],[],[],[],"['Hiphop', 'Watchin', 'Awards', 'Da', 'BET']","['hiphop', 'watchin', 'awards', 'da', 'bet']",[],2012-10-10


In [ ]:
df.head()

,index,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words,date,sentiment_scores
0,61124,394,255819992157786112,"[hiphop, award, bout, live]",250870763,2012-10-10 00:00:13,,,,,[],[],[],[],"[award, live, bout, hiphop]","[award, live, bout, hiphop]",[],2012-10-10,0.984120
1,61125,394,255820118095978496,"[hiphop, award, time]",28026779,2012-10-10 00:00:43,SoundCloud/RaRaSupaStar,,,,[],[],[],[],"[HIPHOP, AWARDS, time]","[hiphop, awards, time]",[],2012-10-10,0.972106
2,61126,394,255820147489636353,"[bet, hiphop, award]",566825483,2012-10-10 00:00:50,,,,,[],[],[],"[(Bet, GPE)]","[award, bet, hiphop]","[award, bet, hiphop]",[],2012-10-10,0.945563
3,61127,394,255820164023595008,"[bet, hiphop, award]",197834311,2012-10-10 00:00:54,Saint Lucia ☀️🌴🇱🇨,,,,[],[],[],[],"[HipHop, BET, award]","[hiphop, bet, award]",[],2012-10-10,0.945563
4,61128,394,255820180884701184,"[watchin, da, bet, hiphop, award]",439490861,2012-10-10 00:00:58,"Michigan, USA",,,,[],[],[],[],"[Hiphop, Watchin, Awards, Da, BET]","[hiphop, watchin, awards, da, bet]",[],2012-10-10,0.982463


In [ ]:
nlp = en_core_web_lg.load()
word_index = tokenize_text(df)[1]
document_features = documents_to_features(df, nlp, word_index)


document_features[0]

2152/2152 [==============================] - 870s 404ms/step


array([ 0.00000000e+00,  1.95272695e-02,  0.00000000e+00,  4.72504318e-01,
        3.08928937e-01,  7.44201466e-02,  0.00000000e+00,  5.92598259e-01,
        0.00000000e+00,  4.21672873e-02,  7.24494815e-01,  9.38764289e-02,
        5.90501487e-01,  9.54401344e-02,  0.00000000e+00,  0.00000000e+00,
        1.61902070e-01,  4.97765541e-02,  1.24185257e-01,  1.02758370e-01,
        0.00000000e+00,  6.90814033e-02,  2.46355534e-01,  1.89586416e-01,
        6.08659023e-03,  1.18976891e-01,  1.57336146e-01,  4.15138930e-01,
        0.00000000e+00,  0.00000000e+00,  2.09431604e-01,  5.04540503e-01,
        2.97841489e-01,  0.00000000e+00,  5.76489031e-01,  4.57861781e-01,
        0.00000000e+00,  2.57708788e-01,  1.26444399e-02,  6.07233942e-01,
        4.67114449e-01,  2.08621204e-01,  0.00000000e+00,  5.55241168e-01,
        4.39872593e-02,  8.36013019e-01,  2.61903614e-01,  6.08828008e-01,
        0.00000000e+00,  2.51102209e-01,  5.66508412e-01,  6.38486683e-01,
        1.75199360e-02,  

In [ ]:
document_features[:10]

array([[0.        , 0.01952727, 0.        , ..., 0.15621346, 0.06896887,
        0.13249469],
       [0.        , 0.27848354, 0.20906352, ..., 0.19425039, 0.0360845 ,
        0.36482394],
       [0.        , 0.27994055, 0.        , ..., 0.24049082, 0.15524535,
        0.11993165],
       ...,
       [0.        , 0.55942434, 0.        , ..., 0.10234042, 0.24486451,
        0.11953337],
       [0.        , 0.26317275, 0.3732276 , ..., 0.13080089, 0.28269607,
        0.28257495],
       [0.        , 0.3066847 , 0.        , ..., 0.13036501, 0.1904665 ,
        0.11334552]], dtype=float32)

In [ ]:
document_features.shape

(68841, 600)

In [ ]:
time_features = get_time_features(df)

print("Time features generated.")

Time features generated.


In [ ]:
time_features[:10]

array([[4.11920000e-01, 1.50462963e-04],
       [4.11920000e-01, 4.97685185e-04],
       [4.11920000e-01, 5.78703704e-04],
       [4.11920000e-01, 6.25000000e-04],
       [4.11920000e-01, 6.71296296e-04],
       [4.11920000e-01, 1.09953704e-03],
       [4.11920000e-01, 1.33101852e-03],
       [4.11920000e-01, 1.51620370e-03],
       [4.11920000e-01, 1.59722222e-03],
       [4.11920000e-01, 1.72453704e-03]])

In [ ]:
time_features.shape

(68841, 2)

In [ ]:
# Concatenate the document features and time features into a single numpy array
combined_features = np.concatenate((document_features, time_features), axis=1)
print("Concatenated document features and time features.")
combined_features.shape

Concatenated document features and time features.


(68841, 602)

In [ ]:
# Save the concatenated features as a numpy array file
save_file_path =  'LSTM_Full_features_69612_0709_spacy_lg_zero_multiclasses_filtered.npy'
np.save(save_file_path, combined_features)
print("Initial features saved.")


Initial features saved.


In [ ]:
# Load the concatenated features from the saved numpy array file
loaded_features = np.load(save_file_path)
print("Initial features loaded.")
print(loaded_features.shape)

Initial features loaded.
(68841, 602)


# CONSTRUCTING GRAPHS

In [ ]:
def construct_graph_from_df(df, G=None):
    if G is None:
        G = nx.Graph()
    for _, row in df.iterrows():
        tid = 't_' + str(row['tweet_id'])
        G.add_node(tid)
        G.nodes[tid]['tweet_id'] = True  # right-hand side value is irrelevant for the lookup

        user_ids = row['user_mentions']
        user_ids  = list(user_ids)
        user_ids.append(row['user_id'])
        user_ids = ['u_' + str(each) for each in user_ids]
        # print(user_ids)
        G.add_nodes_from(user_ids)
        for each in user_ids:
            G._node[each]['user_id'] = True

        entities = row['entities']
        # entities = ['e_' + each for each in entities]
        # print(entities)
        G.add_nodes_from(entities)
        for each in entities:
            G._node[each]['entity'] = True

        words = row['sampled_words']
        words = ['w_' + each for each in words]
        # print(words)
        G.add_nodes_from(words)
        for each in words:
            G._node[each]['word'] = True

        edges = []
        edges += [(tid, each) for each in user_ids]
        edges += [(tid, each) for each in entities]
        edges += [(tid, each) for each in words]
        G.add_edges_from(edges)

    return G


In [ ]:
# convert a heterogeneous social graph G to a homogeneous message graph following eq. 1 of the paper,
# and store the sparse binary adjacency matrix of the homogeneous message graph.


def to_dgl_graph_v3(G, save_path=None):
    message = ''
    print('Start converting heterogeneous networkx graph to homogeneous dgl graph.')
    message += 'Start converting heterogeneous networkx graph to homogeneous dgl graph.\n'
    all_start = time()

    print('\tGetting a list of all nodes ...')
    message += '\tGetting a list of all nodes ...\n'
    start = time()
    all_nodes = list(G.nodes)
    mins = (time() - start) / 60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # print('All nodes: ', all_nodes)
    # print('Total number of nodes: ', len(all_nodes))

    print('\tGetting adjacency matrix ...')
    message += '\tGetting adjacency matrix ...\n'
    start = time()
    A = nx.to_numpy_matrix(G)  # Returns the graph adjacency matrix as a NumPy matrix.
    mins = (time() - start) / 60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # compute commuting matrices
    print('\tGetting lists of nodes of various types ...')
    message += '\tGetting lists of nodes of various types ...\n'
    start = time()
    tid_nodes = list(nx.get_node_attributes(G, 'tweet_id').keys())
    userid_nodes = list(nx.get_node_attributes(G, 'user_id').keys())
    word_nodes = list(nx.get_node_attributes(G, 'word').keys())
    entity_nodes = list(nx.get_node_attributes(G, 'entity').keys())
    del G
    mins = (time() - start) / 60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\tConverting node lists to index lists ...')
    message += '\tConverting node lists to index lists ...\n'
    start = time()
    #  find the index of target nodes in the list of all_nodes
    indices_tid = [all_nodes.index(x) for x in tid_nodes]
    indices_userid = [all_nodes.index(x) for x in userid_nodes]
    indices_word = [all_nodes.index(x) for x in word_nodes]
    indices_entity = [all_nodes.index(x) for x in entity_nodes]
    del tid_nodes
    del userid_nodes
    del word_nodes
    del entity_nodes
    mins = (time() - start) / 60
    print('\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # ----------------------tweet-user-tweet----------------------
    print('\tStart constructing tweet-user-tweet commuting matrix ...')
    print('\t\t\tStart constructing tweet-user matrix ...')
    message += '\tStart constructing tweet-user-tweet commuting matrix ...\n\t\t\tStart constructing tweet-user ' \
               'matrix ...\n '
    start = time()
    w_tid_userid = A[np.ix_(indices_tid, indices_userid)]
    #  return a N(indices_tid)*N(indices_userid) matrix, representing the weight of edges between tid and userid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # convert to scipy sparse matrix
    print('\t\t\tConverting to sparse matrix ...')
    message += '\t\t\tConverting to sparse matrix ...\n'
    start = time()
    s_w_tid_userid = sparse.csr_matrix(w_tid_userid)  # matrix compression
    del w_tid_userid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tTransposing ...')
    message += '\t\t\tTransposing ...\n'
    start = time()
    s_w_userid_tid = s_w_tid_userid.transpose()
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tCalculating tweet-user * user-tweet ...')
    message += '\t\t\tCalculating tweet-user * user-tweet ...\n'
    start = time()
    s_m_tid_userid_tid = s_w_tid_userid * s_w_userid_tid  # homogeneous message graph
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tSaving ...')
    message += '\t\t\tSaving ...\n'
    start = time()
    if save_path is not None:
        sparse.save_npz(save_path + "s_m_tid_userid_tid.npz", s_m_tid_userid_tid)
        print("Sparse binary userid commuting matrix saved.")
        del s_m_tid_userid_tid
    del s_w_tid_userid
    del s_w_userid_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # ----------------------tweet-ent-tweet------------------------
    print('\tStart constructing tweet-ent-tweet commuting matrix ...')
    print('\t\t\tStart constructing tweet-ent matrix ...')
    message += '\tStart constructing tweet-ent-tweet commuting matrix ...\n\t\t\tStart constructing tweet-ent matrix ' \
               '...\n '
    start = time()
    w_tid_entity = A[np.ix_(indices_tid, indices_entity)]
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # convert to scipy sparse matrix
    print('\t\t\tConverting to sparse matrix ...')
    message += '\t\t\tConverting to sparse matrix ...\n'
    start = time()
    s_w_tid_entity = sparse.csr_matrix(w_tid_entity)
    del w_tid_entity
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tTransposing ...')
    message += '\t\t\tTransposing ...\n'
    start = time()
    s_w_entity_tid = s_w_tid_entity.transpose()
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tCalculating tweet-ent * ent-tweet ...')
    message += '\t\t\tCalculating tweet-ent * ent-tweet ...\n'
    start = time()
    s_m_tid_entity_tid = s_w_tid_entity * s_w_entity_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tSaving ...')
    message += '\t\t\tSaving ...\n'
    start = time()
    if save_path is not None:
        sparse.save_npz(save_path + "s_m_tid_entity_tid.npz", s_m_tid_entity_tid)
        print("Sparse binary entity commuting matrix saved.")
        del s_m_tid_entity_tid
    del s_w_tid_entity
    del s_w_entity_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # ----------------------tweet-word-tweet----------------------
    print('\tStart constructing tweet-word-tweet commuting matrix ...')
    print('\t\t\tStart constructing tweet-word matrix ...')
    message += '\tStart constructing tweet-word-tweet commuting matrix ...\n\t\t\tStart constructing tweet-word ' \
               'matrix ...\n '
    start = time()
    w_tid_word = A[np.ix_(indices_tid, indices_word)]
    del A
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # convert to scipy sparse matrix
    print('\t\t\tConverting to sparse matrix ...')
    message += '\t\t\tConverting to sparse matrix ...\n'
    start = time()
    s_w_tid_word = sparse.csr_matrix(w_tid_word)
    del w_tid_word
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tTransposing ...')
    message += '\t\t\tTransposing ...\n'
    start = time()
    s_w_word_tid = s_w_tid_word.transpose()
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tCalculating tweet-word * word-tweet ...')
    message += '\t\t\tCalculating tweet-word * word-tweet ...\n'
    start = time()
    s_m_tid_word_tid = s_w_tid_word * s_w_word_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    print('\t\t\tSaving ...')
    message += '\t\t\tSaving ...\n'
    start = time()
    if save_path is not None:
        sparse.save_npz(save_path + "s_m_tid_word_tid.npz", s_m_tid_word_tid)
        print("Sparse binary word commuting matrix saved.")
        del s_m_tid_word_tid
    del s_w_tid_word
    del s_w_word_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'

    # ----------------------compute tweet-tweet adjacency matrix----------------------
    print('\tComputing tweet-tweet adjacency matrix ...')
    message += '\tComputing tweet-tweet adjacency matrix ...\n'
    start = time()
    if save_path is not None:
        s_m_tid_userid_tid = sparse.load_npz(save_path + "s_m_tid_userid_tid.npz")
        print("Sparse binary userid commuting matrix loaded.")
        s_m_tid_entity_tid = sparse.load_npz(save_path + "s_m_tid_entity_tid.npz")
        print("Sparse binary entity commuting matrix loaded.")
        s_m_tid_word_tid = sparse.load_npz(save_path + "s_m_tid_word_tid.npz")
        print("Sparse binary word commuting matrix loaded.")

    s_A_tid_tid = s_m_tid_userid_tid + s_m_tid_entity_tid
    del s_m_tid_userid_tid
    del s_m_tid_entity_tid
    s_bool_A_tid_tid = (s_A_tid_tid + s_m_tid_word_tid).astype('bool')  # confirm the connect between tweets
    del s_m_tid_word_tid
    del s_A_tid_tid
    mins = (time() - start) / 60
    print('\t\t\tDone. Time elapsed: ', mins, ' mins\n')
    message += '\t\t\tDone. Time elapsed: '
    message += str(mins)
    message += ' mins\n'
    all_mins = (time() - all_start) / 60
    print('\tOver all time elapsed: ', all_mins, ' mins\n')
    message += '\tOver all time elapsed: '
    message += str(all_mins)
    message += ' mins\n'

    if save_path is not None:
        sparse.save_npz(save_path + "s_bool_A_tid_tid.npz", s_bool_A_tid_tid)
        print("Sparse binary adjacency matrix saved.")
        s_bool_A_tid_tid = sparse.load_npz(save_path + "s_bool_A_tid_tid.npz")
        print("Sparse binary adjacency matrix loaded.")

    # create corresponding dgl graph
    G = dgl.DGLGraph(s_bool_A_tid_tid)
    print('We have %d nodes.' % G.number_of_nodes())
    print('We have %d edges.' % G.number_of_edges())
    print()
    message += 'We have '
    message += str(G.number_of_nodes())
    message += ' nodes.'
    message += 'We have '
    message += str(G.number_of_edges())
    message += ' edges.\n'

    return all_mins, message

In [ ]:
def construct_incremental_dataset_0922(df, save_path, features, test=True):
    # If test equals true, construct the initial graph using test_ini_size tweets
    # and increment the graph by test_incr_size tweets each day
    test_ini_size = 500
    test_incr_size = 100

    # save data splits for training/validate/test mask generation
    data_split = []
    # save time spent for the heterogeneous -> homogeneous conversion of each graph
    all_graph_mins = []
    message = ""
    # extract distinct dates
    distinct_dates = df.date.unique()  # 2012-11-07
    # print("Distinct dates: ", distinct_dates)
    print("Number of distinct dates: ", len(distinct_dates))
    print()
    message += "Number of distinct dates: "
    message += str(len(distinct_dates))
    message += "\n"

    # split data by dates and construct graphs
    # first week -> initial graph (20254 tweets)
    print("Start constructing initial graph ...")
    message += "\nStart constructing initial graph ...\n"
    ini_df = df.loc[df['date'].isin(distinct_dates[:7])]  # find top 7 dates
    if test:
        ini_df = ini_df[:test_ini_size]  # top test_ini_size dates
    G = construct_graph_from_df(ini_df)
    path = save_path + '0/'
    os.mkdir(path)
    grap_mins, graph_message = to_dgl_graph_v3(G, save_path=path)
    message += graph_message
    print("Initial graph saved")
    message += "Initial graph saved\n"
    # record the total number of tweets
    data_split.append(ini_df.shape[0])
    # record the time spent for graph conversion
    all_graph_mins.append(grap_mins)
    # extract and save the labels of corresponding tweets
    y = ini_df['event_id'].values
    y = [int(each) for each in y]
    np.save(path + 'labels.npy', np.asarray(y))
    print("Labels saved.")
    message += "Labels saved.\n"
    # extract and save the features of corresponding tweets
    indices = ini_df['index'].values.astype(np.int8).tolist()
    x = features[indices, :]
    np.save(path + 'features.npy', x)
    print("Features saved.")
    message += "Features saved.\n\n"

    # subsequent days -> insert tweets day by day (skip the last day because it only contains one tweet)
    for i in range(7, len(distinct_dates) - 1):
        print("Start constructing graph ", str(i - 6), " ...")
        message += "\nStart constructing graph "
        message += str(i - 6)
        message += " ...\n"
        incr_df = df.loc[df['date'] == distinct_dates[i]]
        if test:
            incr_df = incr_df[:test_incr_size]

        # All/Relevant Message Strategy: keeping all the messages when constructing the graphs
        # (for the Relevant Message Strategy, the unrelated messages will be removed from the graph later on).
        # G = construct_graph_from_df(incr_df, G)

        # Latest Message Strategy: construct graph using only the data of the day
        G = construct_graph_from_df(incr_df)

        path = save_path + str(i - 6) + '/'
        os.mkdir(path)
        grap_mins, graph_message = to_dgl_graph_v3(G, save_path=path)
        message += graph_message
        print("Graph ", str(i - 6), " saved")
        message += "Graph "
        message += str(i - 6)
        message += " saved\n"
        # record the total number of tweets
        data_split.append(incr_df.shape[0])
        # record the time spent for graph conversion
        all_graph_mins.append(grap_mins)
        # extract and save the labels of corresponding tweets
        # y = np.concatenate([y, incr_df['event_id'].values], axis = 0)
        y = [int(each) for each in incr_df['event_id'].values]
        np.save(path + 'labels.npy', y)
        print("Labels saved.")
        message += "Labels saved.\n"
        # extract and save the features of corresponding tweets
        indices = incr_df['index'].values.astype(np.int8).tolist()
        x = features[indices, :]
        # x = np.concatenate([x, x_incr], axis = 0)
        np.save(path + 'features.npy', x)
        print("Features saved.")
        message += "Features saved.\n"

    return message, data_split, all_graph_mins

In [ ]:
len(loaded_features)

68841

In [ ]:
nx.__version__

'2.4'

In [ ]:
# load features
# the dimension of feature is 300 in this dataset

# loaded_features = np.load('features_69612_0709_spacy_lg_zero_multiclasses_filtered.npy')
save_path = 'data_lstm2/'

# generate test graphs, features, and labels
message, data_split, all_graph_mins = construct_incremental_dataset_0922(df, save_path, loaded_features, True)
with open("lstm_full_node_edge_statistics.txt", "w") as text_file:
    text_file.write(message)

np.save('lstm_full_data_split.npy', np.asarray(data_split))
print("Data split: ", data_split)
np.save('lstm_full_all_graph_mins.npy', np.asarray(all_graph_mins))
print("Time sepnt on heterogeneous -> homogeneous graph conversions: ", all_graph_mins)

Number of distinct dates:  29

Start constructing initial graph ...
Start converting heterogeneous networkx graph to homogeneous dgl graph.
	Getting a list of all nodes ...
	Done. Time elapsed:  2.4239222208658853e-06  mins

	Getting adjacency matrix ...
	Done. Time elapsed:  0.0002743363380432129  mins

	Getting lists of nodes of various types ...
	Done. Time elapsed:  3.6092599232991535e-05  mins

	Converting node lists to index lists ...
	Done. Time elapsed:  0.00033576488494873046  mins

	Start constructing tweet-user-tweet commuting matrix ...
			Start constructing tweet-user matrix ...
			Done. Time elapsed:  3.825028737386068e-05  mins

			Converting to sparse matrix ...
			Done. Time elapsed:  0.00014966726303100586  mins

			Transposing ...
			Done. Time elapsed:  3.953774770100911e-06  mins

			Calculating tweet-user * user-tweet ...
			Done. Time elapsed:  0.00021241505940755207  mins

			Saving ...
Sparse binary userid commuting matrix saved.
			Done. Time elapsed:  5.39859

/usr/local/lib/python3.10/dist-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


We have 500 nodes.
We have 11836 edges.

Initial graph saved
Labels saved.
Features saved.
Start constructing graph  1  ...
Start converting heterogeneous networkx graph to homogeneous dgl graph.
	Getting a list of all nodes ...
	Done. Time elapsed:  7.867813110351562e-07  mins

	Getting adjacency matrix ...
	Done. Time elapsed:  1.7182032267252603e-05  mins

	Getting lists of nodes of various types ...
	Done. Time elapsed:  7.871786753336588e-06  mins

	Converting node lists to index lists ...
	Done. Time elapsed:  2.1620591481526694e-05  mins

	Start constructing tweet-user-tweet commuting matrix ...
			Start constructing tweet-user matrix ...
			Done. Time elapsed:  3.7749608357747397e-06  mins

			Converting to sparse matrix ...
			Done. Time elapsed:  8.79367192586263e-06  mins

			Transposing ...
			Done. Time elapsed:  2.392133076985677e-06  mins

			Calculating tweet-user * user-tweet ...
			Done. Time elapsed:  1.7182032267252603e-05  mins

			Saving ...
Sparse binary userid c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
